# Baran Experiment

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas
import IPython.display

import raha
import datetime

In [3]:
def format_delta(delta: datetime.timedelta):
    hours, remainder = divmod(delta.total_seconds(), 3600)
    minutes, seconds = divmod(remainder, 60)
    
    h = f'{int(hours)}h ' if hours > 0 else ''
    m = f'{int(minutes)}m ' if minutes > 0 else ''
    s = f'{int(seconds)}s'
    
    return h+m+s

def estimate_time_to_finish(times: list, current_run_i: int, total_runs: int):
    deltas = []
    i = 1
    
    while i+1 <= len(times):
        deltas.append(times[i] - times[i-1])
        i += 1
        
    avg = sum(deltas, datetime.timedelta())/len(deltas)
    return f'Run {current_run_i}/{total_runs}. {format_delta(avg)} per run, estimate {format_delta(avg*(total_runs - current_run_i))} to finish'

In [4]:
beers = {
    "name": "beers",
    "path": "../datasets/beers/dirty.csv",
    "clean_path": "../datasets/beers/clean.csv"
}
flights = {
    "name": "flights",
    "path": "../datasets/flights/dirty.csv",
    "clean_path": "../datasets/flights/clean.csv"
}
movies = {
    "name": "movies",
    "path": "../datasets/movies_1/dirty.csv",
    "clean_path": "../datasets/movies_1/clean.csv"
}
rayyan = {
    "name": "rayyan",
    "path": "../datasets/rayyan/dirty.csv",
    "clean_path": "../datasets/rayyan/clean.csv"
}
tax = {
    "name": "tax",
    "path": "../datasets/tax/dirty.csv",
    "clean_path": "../datasets/tax/clean.csv"
}
toy = {
    "name": "toy",
    "path": "../datasets/toy/dirty.csv",
    "clean_path": "../datasets/toy/clean.csv"
}
hospital = {
    "name": "hospital",
    "path": "../datasets/hospital/dirty.csv",
    "clean_path": "../datasets/hospital/clean.csv"
}

In [7]:
datasets = [hospital]
experiments = ['adder']
runs = range(10)
duration_experiment = len(datasets) * len(runs) * len(experiments)

results = []
times = [datetime.datetime.now()]
i = 1
for dataset_dictionary in datasets:
    for run in runs:
        for e in experiments:            
            app_2 = raha.Correction()
            print(f'Starting run {i}/{duration_experiment}.')

            app_2.LABELING_BUDGET = 20
            app_2.VERBOSE = False
            app_2.CLASSIFICATION_MODEL = "ABC"
            app_2.EXPERIMENT = e
            app_2.IMPUTER_FEATURE_GENERATOR = True

            d = raha.Dataset(dataset_dictionary)
            d.detected_cells = dict(d.get_actual_errors_dictionary())
            d = app_2.initialize_dataset(d)
            
            app_2.initialize_models(d)
            while len(d.labeled_tuples) < app_2.LABELING_BUDGET:
                app_2.sample_tuple(d, random_seed=run)
                if d.has_ground_truth:
                    app_2.label_with_ground_truth(d)
            app_2.update_models(d)
            app_2.generate_features(d)
            app_2.predict_corrections(d, random_seed=run)
                
            p, r, f = d.get_data_cleaning_evaluation(d.corrected_cells)[-3:]
            results.append({'dataset': d.name, 'experiment': e, 'run': run, 'precision': p, 'recall': r, 'f1': f})
            
            # time estimates for the measurement
            times.append(datetime.datetime.now())
            print(estimate_time_to_finish(times, i, duration_experiment))
            i += 1

Starting run 1/10.
Failed to train an imputer model for column 0
<class 'datawig.autogluon_imputer.TargetColumnException'>


		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		"None of [Index(['NeuralNetFastAI_0_0'], dtype='object')] are in the [columns]"
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		"None of [Index(['NeuralNetFastAI_0_0'], dtype='object')] are in the [columns]"


Run 1/10. 10m 40s per run, estimate 1h 36m 0s to finish
Starting run 2/10.
Failed to train an imputer model for column 0
<class 'datawig.autogluon_imputer.TargetColumnException'>


		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		"None of [Index(['NeuralNetFastAI_0_0'], dtype='object')] are in the [columns]"
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		"None of [Index(['NeuralNetFastAI_0_0'], dtype='object')] are in the [columns]"


Run 2/10. 10m 0s per run, estimate 1h 20m 4s to finish
Starting run 3/10.
Failed to train an imputer model for column 0
<class 'datawig.autogluon_imputer.TargetColumnException'>


...

Run 8/10. 9m 18s per run, estimate 18m 37s to finish
Starting run 9/10.
Failed to train an imputer model for column 0
<class 'datawig.autogluon_imputer.TargetColumnException'>


		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		"None of [Index(['NeuralNetFastAI_0_0'], dtype='object')] are in the [columns]"
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		"None of [Index(['NeuralNetFastAI_0_0'], dtype='object')] are in the [columns]"


Run 9/10. 9m 17s per run, estimate 9m 17s to finish
Starting run 10/10.
Failed to train an imputer model for column 0
<class 'datawig.autogluon_imputer.TargetColumnException'>


		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		"None of [Index(['NeuralNetFastAI_0_0'], dtype='object')] are in the [columns]"
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		Unknown y_pred_proba format for `problem_type="multiclass"`.
		"None of [Index(['NeuralNetFastAI_0_0'], dtype='object')] are in the [columns]"


Run 10/10. 9m 14s per run, estimate 0s to finish


In [11]:
print(results)

[{'dataset': 'hospital', 'experiment': 'adder', 'run': 0, 'precision': 0.9613636363636363, 'recall': 0.831041257367387, 'f1': 0.8914646996838776}, {'dataset': 'hospital', 'experiment': 'adder', 'run': 1, 'precision': 0.9563218390804598, 'recall': 0.8172888015717092, 'f1': 0.8813559322033899}, {'dataset': 'hospital', 'experiment': 'adder', 'run': 2, 'precision': 0.9298642533936652, 'recall': 0.8074656188605108, 'f1': 0.8643533123028393}, {'dataset': 'hospital', 'experiment': 'adder', 'run': 3, 'precision': 0.9176201372997712, 'recall': 0.787819253438114, 'f1': 0.8477801268498943}, {'dataset': 'hospital', 'experiment': 'adder', 'run': 4, 'precision': 0.8584474885844748, 'recall': 0.7387033398821218, 'f1': 0.7940865892291445}, {'dataset': 'hospital', 'experiment': 'adder', 'run': 5, 'precision': 0.9168591224018475, 'recall': 0.7799607072691552, 'f1': 0.842887473460722}, {'dataset': 'hospital', 'experiment': 'adder', 'run': 6, 'precision': 0.9614512471655329, 'recall': 0.8330058939096268, 

In [10]:

sum([x['f1'] for x in results])/len(results)

0.8647948241956177

In [6]:
print(results)  # to be done; 90s training time

[{'dataset': 'hospital', 'experiment': 'adder', 'run': 9, 'precision': 0.9588100686498856, 'recall': 0.8231827111984283, 'f1': 0.8858350951374208}]


In [6]:
print(results)  # 30s training time

[{'dataset': 'hospital', 'experiment': 'adder', 'run': 9, 'precision': 0.9130434782608695, 'recall': 0.825147347740668, 'f1': 0.8668730650154799}]


In [22]:
print(results)   # no imputer model

[{'dataset': 'hospital', 'experiment': 'adder', 'run': 9, 'precision': 0.9551569506726457, 'recall': 0.8369351669941061, 'f1': 0.8921465968586387}]


In [20]:
old_pair_features = d.pair_features

In [46]:
app_2.predict_corrections(d, random_seed=run)
p, r, f = d.get_data_cleaning_evaluation(d.corrected_cells)[-3:]

In [48]:
f

0.8898216159496327

In [29]:
datasets = [beers, 
            hospital,
            rayyan,
            flights]
experiments = ['pdep', 'adder', 'constant', 'ente', 'disable_vicinity']
runs = range(3)
duration_experiment = len(datasets) * len(runs) * len(experiments)

results = []
times = [datetime.datetime.now()]
i = 1
for dataset_dictionary in datasets:
    for run in runs:
        for e in experiments:            
            app_2 = raha.Correction()
            print(f'Starting run {i}/{duration_experiment}.')

            app_2.LABELING_BUDGET = 20
            app_2.VERBOSE = False
            app_2.CLASSIFICATION_MODEL = "DTC"
            app_2.EXPERIMENT = e

            d = raha.dataset.Dataset(dataset_dictionary, n_rows=500)
            d.detected_cells = dict(d.get_actual_errors_dictionary())
            d = app_2.initialize_dataset(d)
            
            app_2.initialize_models(d)
            while len(d.labeled_tuples) < app_2.LABELING_BUDGET:
                app_2.sample_tuple(d, random_seed=run)
                if d.has_ground_truth:
                    app_2.label_with_ground_truth(d)
                app_2.update_models(d)
                app_2.generate_features(d)
                app_2.predict_corrections(d, random_seed=run)
                
            p, r, f = d.get_data_cleaning_evaluation(d.corrected_cells)[-3:]
            results.append({'dataset': d.name, 'experiment': e, 'run': run, 'precision': p, 'recall': r, 'f1': f})
            
            # time estimates for the measurement
            times.append(datetime.datetime.now())
            print(estimate_time_to_finish(times, i, duration_experiment))
            i += 1

Starting run 1/60.


Process ForkPoolWorker-31:
Process ForkPoolWorker-35:
Process ForkPoolWorker-28:
Process ForkPoolWorker-36:
Process ForkPoolWorker-33:
Process ForkPoolWorker-34:


KeyboardInterrupt: 

Process ForkPoolWorker-29:
Process ForkPoolWorker-27:
Process ForkPoolWorker-32:
Process ForkPoolWorker-26:
Process ForkPoolWorker-25:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/philipp/.pyenv/versions/3.7.10/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/philipp/.pyenv/versions/3.7.10/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/philipp/.pyenv/versions/3.7.10/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/philipp/.pyenv/versions/3.7.10/lib/python3.7/multiprocessing/process.py", line 29

In [7]:
print(results)

[{'dataset': 'beers', 'experiment': 'pdep', 'run': 0, 'precision': 0.8536126147328783, 'recall': 0.8314993122420908, 'f1': 0.842410869817675}, {'dataset': 'beers', 'experiment': 'adder', 'run': 0, 'precision': 0.8489056248529065, 'recall': 0.8269142595139845, 'f1': 0.837765648589014}, {'dataset': 'beers', 'experiment': 'constant', 'run': 0, 'precision': 0.849105461393597, 'recall': 0.8269142595139845, 'f1': 0.8378629500580721}, {'dataset': 'beers', 'experiment': 'ente', 'run': 0, 'precision': 0.8494921514312096, 'recall': 0.8436497019715726, 'f1': 0.8465608465608465}, {'dataset': 'beers', 'experiment': 'disable_vicinity', 'run': 0, 'precision': 0.8541089566020313, 'recall': 0.848234754699679, 'f1': 0.8511617207269381}, {'dataset': 'beers', 'experiment': 'pdep', 'run': 1, 'precision': 0.84375, 'recall': 0.8356258596973866, 'f1': 0.8396682791983414}, {'dataset': 'beers', 'experiment': 'adder', 'run': 1, 'precision': 0.6898148148148148, 'recall': 0.6831728564878496, 'f1': 0.68647777009905